In [ ]:
!pip install pillow
!pip install opencv-python

In [ ]:
import os
import urllib.request

# Create the 'models' directory if it doesn't exist
if not os.path.exists('model'):
    os.makedirs('model')

# Download the model files
model_files = [
    ('colorization_deploy_v2.prototxt', 'https://raw.githubusercontent.com/richzhang/colorization/caffe/colorization/models/colorization_deploy_v2.prototxt'),
    ('pts_in_hull.npy', 'https://raw.githubusercontent.com/richzhang/colorization/caffe/colorization/resources/pts_in_hull.npy'),
    ('colorization_release_v2.caffemodel', 'https://www.dropbox.com/s/dx0qvhhp5hbcx7z/colorization_release_v2.caffemodel?dl=1')
]

for filename, url in model_files:
    filepath = os.path.join('model', filename)
    try:
        urllib.request.urlretrieve(url, filepath)
        print(f"Downloaded {filename} to {filepath}")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")

Downloaded colorization_deploy_v2.prototxt to model/colorization_deploy_v2.prototxt
Downloaded pts_in_hull.npy to model/pts_in_hull.npy
Downloaded colorization_release_v2.caffemodel to model/colorization_release_v2.caffemodel


In [ ]:
import cv2
import numpy as np
import os
import ffmpeg
from google.colab.patches import cv2_imshow

# Load the colorization model
prototxt_path = 'model/colorization_deploy_v2.prototxt'
model_path = 'model/colorization_release_v2.caffemodel'
cluster_centers_path = 'model/pts_in_hull.npy'

net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
pts_in_hull = np.load(cluster_centers_path)
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1).astype(np.float32)
class8 = net.getLayerId('class8_ab')
conv8 = net.getLayerId('conv8_313_rh')
net.getLayer(class8).blobs = [pts_in_hull]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [ ]:
# Function to colorize a frame
def colorize_frame(frame):
    print("Colorizing frame...")
    scaled = frame.astype("float32") / 255.0
    lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
    L = lab[:, :, 0]
    L_resized = cv2.resize(L, (224, 224))
    L_resized -= 50
    net.setInput(cv2.dnn.blobFromImage(L_resized))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    ab_resized = cv2.resize(ab, (frame.shape[1], frame.shape[0]))
    L = lab[:, :, 0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab_resized), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = (colorized * 255).astype("uint8")
    return colorized


In [ ]:
# Step 1: Upload a video
from google.colab import files
uploaded = files.upload()

input_video_path = list(uploaded.keys())[0]  # Get the uploaded file name

# Step 2: Separate audio from video
def extract_audio(input_video, output_audio):
    ffmpeg.input(input_video).output(output_audio).run()

audio_path = 'output_audio.aac'
extract_audio(input_video_path, audio_path)

# Step 3: Convert video into frames
def video_to_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(f"{output_folder}/frame_{count:05d}.jpg", frame)
        count += 1
    cap.release()

frame_folder = 'frames'
video_to_frames(input_video_path, frame_folder)

# Step 4: Apply colorization to each frame
def colorize_frames(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_files = sorted(os.listdir(input_folder))
    for frame_file in frame_files:
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path)
        colorized_frame = colorize_frame(frame)
        output_frame_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_frame_path, colorized_frame)

colorized_frame_folder = 'colorized_frames'
colorize_frames(frame_folder, colorized_frame_folder)

# Step 5: Reconstruct the video from the colorized frames
def frames_to_video(frame_folder, output_video_path, fps):
    frame_files = sorted([os.path.join(frame_folder, img) for img in os.listdir(frame_folder)])
    frame = cv2.imread(frame_files[0])
    height, width, layers = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        out.write(frame)
    out.release()

output_video_path = 'colorized_video.mp4'
frames_to_video(colorized_frame_folder, output_video_path, fps=24)

# Step 6: Add audio to the video
def add_audio_to_video(video_path, audio_path, output_path):
    input_video = ffmpeg.input(video_path)
    input_audio = ffmpeg.input(audio_path)
    ffmpeg.output(input_video, input_audio, output_path).run()


In [ ]:
final_video_path = 'final_video_with_audio.mp4'
add_audio_to_video(output_video_path, audio_path, final_video_path)

print(f"Video saved at {final_video_path}")